In [45]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Set path to the main project directory
path = r'C:\Users\sudee\OneDrive\Documents\Python Scripts\Instacart Basket Analysis\02 Data'

# Load datasets
df_ords = pd.read_csv(os.path.join(path, 'Prepared Data', 'orders_checked_clean.csv'))
df_prods = pd.read_csv(os.path.join(path, 'Prepared Data', 'products_checked_clean.csv'))
df_ords_prior = pd.read_csv(os.path.join(path, 'Prepared Data', 'orders_products_combined.csv'))

# Check the shapes of the datasets
print('df_ords shape:', df_ords.shape)
print('df_prods shape:', df_prods.shape)
print('df_ords_prior shape:', df_ords_prior.shape)




df_ords shape: (3421083, 6)
df_prods shape: (49688, 5)
df_ords_prior shape: (6, 13)


In [47]:
# Merge the datasets
df_merged = pd.merge(df_ords_prior, df_ords, on='order_id', how='inner')
df_merged_large = pd.merge(df_merged, df_prods, on='product_id', how='inner')

# Check the shape and columns of the merged dataframe
print("df_merged_large shape:", df_merged_large.shape)
print("Columns in df_merged_large:", df_merged_large.columns)


df_merged_large shape: (6, 22)
Columns in df_merged_large: Index(['order_id', 'user_id_x', 'order_number_x', 'order_day_of_week_x',
       'order_hour_of_day_x', 'days_since_prior_order_x', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name_x', 'aisle_id_x',
       'department_id_x', 'prices_x', 'user_id_y', 'order_number_y',
       'order_day_of_week_y', 'order_hour_of_day_y',
       'days_since_prior_order_y', 'product_name_y', 'aisle_id_y',
       'department_id_y', 'prices_y'],
      dtype='object')


In [49]:
# Rename columns to remove duplication
df_merged_large.rename(columns={
    'user_id_x': 'user_id',
    'order_number_x': 'order_number',
    'order_day_of_week_x': 'order_day_of_week',
    'order_hour_of_day_x': 'order_hour_of_day',
    'days_since_prior_order_x': 'days_since_prior_order',
    'product_name_x': 'product_name',
    'aisle_id_x': 'aisle_id',
    'department_id_x': 'department_id',
    'prices_x': 'prices'
}, inplace=True)

# Drop duplicate columns
df_merged_large.drop(columns=[
    'user_id_y', 'order_number_y', 'order_day_of_week_y',
    'order_hour_of_day_y', 'days_since_prior_order_y',
    'product_name_y', 'aisle_id_y', 'department_id_y', 'prices_y'
], inplace=True)

# Check the shape and columns of the cleaned dataframe
print("df_merged_large shape after renaming and dropping duplicates:", df_merged_large.shape)
print("Columns in df_merged_large after renaming and dropping duplicates:", df_merged_large.columns)


df_merged_large shape after renaming and dropping duplicates: (6, 13)
Columns in df_merged_large after renaming and dropping duplicates: Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices'],
      dtype='object')


In [51]:
# Deriving a new column for price range using loc
df_merged_large.loc[df_merged_large['prices'] <= 5, 'price_range'] = 'Low-range product'
df_merged_large.loc[(df_merged_large['prices'] > 5) & (df_merged_large['prices'] <= 15), 'price_range'] = 'Mid-range product'
df_merged_large.loc[df_merged_large['prices'] > 15, 'price_range'] = 'High-range product'


In [53]:
# Deriving a new column for loyalty flag using transform and loc
df_merged_large['max_order'] = df_merged_large.groupby(['user_id'])['order_number'].transform('max')
df_merged_large.loc[df_merged_large['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_merged_large.loc[(df_merged_large['max_order'] <= 40) & (df_merged_large['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_merged_large.loc[df_merged_large['max_order'] <= 10, 'loyalty_flag'] = 'New customer'


In [55]:
# Deriving a new column for spending flag using transform and loc
df_merged_large['avg_price'] = df_merged_large.groupby(['user_id'])['prices'].transform('mean')
df_merged_large.loc[df_merged_large['avg_price'] < 10, 'spending_flag'] = 'Low spender'
df_merged_large.loc[df_merged_large['avg_price'] >= 10, 'spending_flag'] = 'High spender'


In [57]:
# Deriving a new column for order frequency flag using transform and loc
df_merged_large['median_days_since_prior_order'] = df_merged_large.groupby(['user_id'])['days_since_prior_order'].transform('median')
df_merged_large.loc[df_merged_large['median_days_since_prior_order'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
df_merged_large.loc[(df_merged_large['median_days_since_prior_order'] > 10) & (df_merged_large['median_days_since_prior_order'] <= 20), 'order_frequency_flag'] = 'Regular customer'
df_merged_large.loc[df_merged_large['median_days_since_prior_order'] <= 10, 'order_frequency_flag'] = 'Frequent customer'


In [59]:
# Check the shape and columns of the final dataframe
print("df_merged_large shape:", df_merged_large.shape)
print("Columns in df_merged_large:", df_merged_large.columns)

# Display first few rows of the final dataframe
print(df_merged_large.head())


df_merged_large shape: (6, 20)
Columns in df_merged_large: Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range', 'max_order', 'loyalty_flag',
       'avg_price', 'spending_flag', 'median_days_since_prior_order',
       'order_frequency_flag'],
      dtype='object')
   order_id  user_id  order_number  order_day_of_week  order_hour_of_day  \
0         1   112108             4                  4                 10   
1         1   112108             4                  4                 10   
2         2   202279             3                  5                  9   
3         2   202279             3                  5                  9   
4         3   205970            16                  5                 17   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0      

In [61]:
# Export the final dataframe as a pickle file
df_merged_large.to_pickle(os.path.join(path, 'orders_products_combined_updated.pkl'))


In [63]:
# Frequency of price_range
price_range_freq = df_merged_large['price_range'].value_counts().reset_index()
price_range_freq.columns = ['price_range', 'frequency']
print("Frequency of price_range:")
print(price_range_freq)


Frequency of price_range:
         price_range  frequency
0  Mid-range product          5
1  Low-range product          1


In [65]:
# Frequency of loyalty_flag
loyalty_flag_freq = df_merged_large['loyalty_flag'].value_counts().reset_index()
loyalty_flag_freq.columns = ['loyalty_flag', 'frequency']
print("Frequency of loyalty_flag:")
print(loyalty_flag_freq)


Frequency of loyalty_flag:
       loyalty_flag  frequency
0      New customer          4
1  Regular customer          2


In [67]:
# Frequency of spending_flag
spending_flag_freq = df_merged_large['spending_flag'].value_counts().reset_index()
spending_flag_freq.columns = ['spending_flag', 'frequency']
print("Frequency of spending_flag:")
print(spending_flag_freq)


Frequency of spending_flag:
  spending_flag  frequency
0   Low spender          6


In [69]:
# Frequency of order_frequency_flag
order_frequency_flag_freq = df_merged_large['order_frequency_flag'].value_counts().reset_index()
order_frequency_flag_freq.columns = ['order_frequency_flag', 'frequency']
print("Frequency of order_frequency_flag:")
print(order_frequency_flag_freq)


Frequency of order_frequency_flag:
  order_frequency_flag  frequency
0    Frequent customer          4
1     Regular customer          2
